In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install requests
!pip install html5lib
!pip install bs4
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import selenium
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup

In [ ]:
!pip install nest-asyncio
!pip install aiohttp==3.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 751 kB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 271 kB 71.9 MB/s 
     |████████████████████████████████| 149 kB 58.5 MB/s 
  Created wheel for aiohttp: filename=aiohttp-3.1.1-cp37-cp37m-linux_x86_64.whl size=644671 sha256=1ce0933a3a1e867801c97f50930adfd049600d2217436b51eb5f759cb5d7d2c8
  Stored in directory: /root/.cache/pip/wheels/c4/1b/cb/ba0aa4c5733ecfd05e69ed3b6d529cd4554600b44777504d1e
Successfully built aiohttp


In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
# with open('/content/drive/MyDrive/ILSUM/Data/test.txt', 'w') as f:
#     f.write(json.dumps(urls))
import json
#Now read the file back into a Python list object
with open('/content/drive/MyDrive/ILSUM/Data/test.txt', 'r') as f:
    a = json.loads(f.read())

len(a)

29000

In [ ]:
#urls=a
for i in range(29000):
  if urls[i].startswith('https')==False:
    urls[i]='https://gujarati.news18.com/'+urls[i]
    print(i)


In [ ]:
di=set(urls)
urls=list(di)
len(urls)

27327

In [ ]:
import aiohttp
import json
import asyncio
from aiohttp import ClientSession
import requests
from bs4 import BeautifulSoup as bs

async def fetch(url):
  async with ClientSession() as session:
    async with session.get(url) as response:
      try:
        a=await response.text()
        soup = bs(a,"html.parser")
        scripts = soup.find_all('script', attrs={'type': 'application/ld+json'})
        script = scripts[1]
        j_obj = {
            'Link':json.loads(scripts[1].contents[0],strict=False)['url'],
            'Heading':json.loads(scripts[1].contents[0],strict=False)['headline'],
            'Subheading':json.loads(scripts[1].contents[0],strict=False)['description'],
            'Article':json.loads(scripts[1].contents[0],strict=False)['articleBody']
        }
        return j_obj

      except Exception:
        return {}


def fetch_all(url):
    loop = asyncio.get_event_loop()

    a=loop.run_until_complete(fetch(url))
    return a


In [ ]:
import pandas as pd
from multiprocessing import Pool
if __name__=='__main__':

    p = Pool(50)
    data = p.map(fetch_all, list(urls[6000:7000]))
    content=pd.DataFrame(data)
    p.close()
    p.join()
# df= pd.DataFrame(columns=['Link','Heading','Subheading','Article'])
# for i in range(len(urls)):
#   df.loc[len(df.index)] = fetch_all(urls[i])

In [ ]:
content.dropna(inplace=True)
content.reset_index(inplace = True, drop = True)
content

,Link,Heading,Subheading,Article
0,https://gujarati.news18.com/news/gujarat/kutch...,ભાવનગર: સરકારી શાળામાં ધો.1થી 5ના વર્ગો કરાયા ...,Bhavnagar News: ભાવનગરના હાદાનગર વિસ્તારમાં સર...,ભાવનગર: કોરોના મહામારીના (Corona Pandemic) કા...
1,https://gujarati.news18.com/news/gujarat/south...,"સુરતમાં પ્રસૂતા સાથે તબીબની ઘોર બેદરકારી, મૃતક...",Surat news: આ કેસમાં ડોક્ટરના વર્તનમાં વિરોધાભ...,સુરત: ડોક્ટરને ( Doctor) ભગવાનનું સ્વરૂપ માનવા...
2,https://gujarati.news18.com/news/gujarat/kutch...,જામનગર: પ્રેમિકાનું લગ્ન ન થાય તે માટે બોયફ્રે...,Jamnagar News: પોલીસે આરોપીના ફોનના લોકેશનની મ...,જામનગર: જામજોધપુરમાં એક ચોંકાવનારો કિસ્સો સામે...
3,https://gujarati.news18.com/photogallery/gujar...,ગીર: 10 વર્ષના બાળકે શિકાર બનાવવા જતા 14 ફૂટના...,"14 ફૂટ લાંબા અજગર, આશીષને પોતાનો શિકાર બનાવે ત...",ગીર: માળિયા હાટિના તાલુકાના જલંધર ગામની સીમમા...
4,https://gujarati.news18.com/photogallery/gujar...,"કડોદરા GIDCની કંપનીમાં આગ લાગતા બેના મોત, 100થ...",surat News: આગ લાગતા જ સુરત શહેર અને જિલ્લાની ...,સુરત: કડોદરા GIDCની (Kadodara GIDC) મિલમાં સવ...
...,...,...,...,...
681,https://gujarati.news18.com/news/rajkot/detent...,"રાજકોટ : PCR વાનમાં જ પોલીસ કર્મીની રંગરલીયા, ...","રાજકોટ ગ્રામ્ય પોલીસનો વિડીયો વાઇરલ, રાહદારીઓએ...",રાજકોટ : ગ્રામ્ય પોલીસનો વિડીયો વાઇરલ થયો છે. ...
682,https://gujarati.news18.com/photogallery/madhy...,"સોલા સિવિલ હૉસ્પિટલનું સ્વાસ્થ્ય 'મરણ પથારીએ,'...",Sola Civil Hospital Ahmedabad : અમદાવાદની સોલા...,\r\n\r\n સંજય ટાંક અમદાવાદ : અમદાવાદમાં રાજ્...
683,https://gujarati.news18.com/news/gujarat/madhy...,વડોદરા: કેતન ઇનામદારને ન મળી બરોડા ડેરી સામે ધ...,Vadodara News: 'બરોડા ડેરીના સંચાલકો પૈસાના જો...,વડોદરા : પશુપાલકોને બરોડા ડેરી (Baroda Dairy) ...
684,https://gujarati.news18.com/photogallery/north...,"બનાસકાંઠા: અમીરગઢ પાસે દૂધના ટેન્કર, બસ અને ટ્...",Banaskantha road accident: પાલનપુર- આબુ રોડ ને...,"આનંદ જયસ્વાલ, બનાસકાંઠા: બનાસકાંઠામાં અમીરગઢ ..."


In [ ]:
content.to_csv('/content/drive/MyDrive/ILSUM/Data/6-7.csv', index=False)